In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))
sys.path.append(os.path.join(os.path.abspath(os.path.join(os.getcwd(), "..", "..")), "app"))

In [2]:
import json
import pandas as pd
from sqlalchemy import insert
from sqlalchemy.exc import IntegrityError, SQLAlchemyError
from app.database import SessionLocal
from app.models import MajorCategory, Category

In [3]:
db = SessionLocal()

### major_category && category

In [4]:
with open('data_classified_food.json', "r", encoding="utf-8") as f:
    data = json.load(f)
data

{'쌀/면/떡': {'쌀류': {'194629': '백미',
   '194632': '현미',
   '194633': '찹쌀',
   '194634': '흑미',
   '194637': '기능성쌀',
   '493087': '곤약쌀'},
  '면류': {'486616': '봉지라면',
   '486618': '짜장라면',
   '486619': '짬뽕라면',
   '486620': '비빔라면',
   '486622': '컵라면',
   '432577': '짜장면',
   '432578': '짬뽕',
   '432579': '우동',
   '432580': '스파게티',
   '432583': '칼국수',
   '432584': '쌀국수',
   '432586': '냉면',
   '432587': '비빔냉면',
   '432590': '쫄면',
   '432593': '수제비',
   '486443': '콩국수',
   '486445': '월남쌈',
   '486846': '즉석파스타',
   '486848': '페투치네',
   '486849': '링귀니',
   '486850': '펜네',
   '486851': '파르펠리',
   '486852': '리가토니',
   '486475': '물냉면',
   '486845': '당면',
   '432821': '면'},
  '떡류': {'432790': '떡',
   '432791': '가래떡',
   '432793': '찹쌀떡',
   '432794': '두텁떡',
   '432795': '콩떡',
   '432796': '감자떡',
   '432797': '송편',
   '432798': '백설기',
   '432799': '경단',
   '432800': '오메기떡',
   '432801': '떡케익',
   '225499': '떡볶이떡',
   '225500': '떡국떡',
   '446062': '떡케이크'},
  '곡류/두류': {'194642': '콩',
   '194646': '강낭콩',
   '1

In [5]:
# 대분류
major_categories = list(data.keys())
major_categories

['쌀/면/떡', '육류/해산물/계란', '채소/과일', '음료/유제품', '양념/소스/조미료', '가공/편의식/스낵']

In [6]:
# 대분류 insert
data_to_insert = [{"name": m_cat} for m_cat in major_categories]
try:
    stmt = insert(MajorCategory)
    db.execute(stmt, data_to_insert)
    db.commit()

except IntegrityError as e:
    db.rollback()
    print(f"DB Integrity Error: {e}")

except SQLAlchemyError as e:
    db.rollback()
    print(f"DB Error: {e}")

except Exception as e:
    print(f"Error: {e}")

In [7]:
# 소분류 인덱싱화
indexed_categories = [{i + 1: v} for i, (k, v) in enumerate(data.items())]
indexed_categories

[{1: {'쌀류': {'194629': '백미',
    '194632': '현미',
    '194633': '찹쌀',
    '194634': '흑미',
    '194637': '기능성쌀',
    '493087': '곤약쌀'},
   '면류': {'486616': '봉지라면',
    '486618': '짜장라면',
    '486619': '짬뽕라면',
    '486620': '비빔라면',
    '486622': '컵라면',
    '432577': '짜장면',
    '432578': '짬뽕',
    '432579': '우동',
    '432580': '스파게티',
    '432583': '칼국수',
    '432584': '쌀국수',
    '432586': '냉면',
    '432587': '비빔냉면',
    '432590': '쫄면',
    '432593': '수제비',
    '486443': '콩국수',
    '486445': '월남쌈',
    '486846': '즉석파스타',
    '486848': '페투치네',
    '486849': '링귀니',
    '486850': '펜네',
    '486851': '파르펠리',
    '486852': '리가토니',
    '486475': '물냉면',
    '486845': '당면',
    '432821': '면'},
   '떡류': {'432790': '떡',
    '432791': '가래떡',
    '432793': '찹쌀떡',
    '432794': '두텁떡',
    '432795': '콩떡',
    '432796': '감자떡',
    '432797': '송편',
    '432798': '백설기',
    '432799': '경단',
    '432800': '오메기떡',
    '432801': '떡케익',
    '225499': '떡볶이떡',
    '225500': '떡국떡',
    '446062': '떡케이크'},
   '곡류/두류': 

In [8]:
# 소분류 data_to_insert
data_insert = list()
data_to_insert = list()
for i_cat in indexed_categories:
    for k, v in i_cat.items():
        for vi_k, vi_v in v.items():
            data_insert.append({"major_category_id": k, "name": vi_k, "products": vi_v})
            data_to_insert.append({"major_category_id": k, "name": vi_k})
data_to_insert

[{'major_category_id': 1, 'name': '쌀류'},
 {'major_category_id': 1, 'name': '면류'},
 {'major_category_id': 1, 'name': '떡류'},
 {'major_category_id': 1, 'name': '곡류/두류'},
 {'major_category_id': 1, 'name': '가루류'},
 {'major_category_id': 1, 'name': '즉석식품'},
 {'major_category_id': 2, 'name': '소고기'},
 {'major_category_id': 2, 'name': '돼지고기'},
 {'major_category_id': 2, 'name': '닭/오리/기타'},
 {'major_category_id': 2, 'name': '해산물'},
 {'major_category_id': 2, 'name': '계란'},
 {'major_category_id': 2, 'name': '두부/유부'},
 {'major_category_id': 3, 'name': '잎채소'},
 {'major_category_id': 3, 'name': '근채류'},
 {'major_category_id': 3, 'name': '과채류'},
 {'major_category_id': 3, 'name': '버섯류'},
 {'major_category_id': 3, 'name': '과일'},
 {'major_category_id': 3, 'name': '냉동과일'},
 {'major_category_id': 3, 'name': '채소/기타'},
 {'major_category_id': 4, 'name': '생수/탄산수'},
 {'major_category_id': 4, 'name': '음료/전통주'},
 {'major_category_id': 4, 'name': '우유'},
 {'major_category_id': 4, 'name': '두유'},
 {'major_category_id':

In [9]:
# 소분류 insert
try:
    stmt = insert(Category)
    db.execute(stmt, data_to_insert)
    db.commit()

except IntegrityError as e:
    db.rollback()
    print(f"DB Integrity Error: {e}")

except SQLAlchemyError as e:
    db.rollback()
    print(f"DB Error: {e}")

except Exception as e:
    print(f"Error: {e}")

### product

In [10]:
category_items = list()
i = 1
for data in data_insert:
    category_items.append({i: data['products']})
    i += 1
category_items

[{1: {'194629': '백미',
   '194632': '현미',
   '194633': '찹쌀',
   '194634': '흑미',
   '194637': '기능성쌀',
   '493087': '곤약쌀'}},
 {2: {'486616': '봉지라면',
   '486618': '짜장라면',
   '486619': '짬뽕라면',
   '486620': '비빔라면',
   '486622': '컵라면',
   '432577': '짜장면',
   '432578': '짬뽕',
   '432579': '우동',
   '432580': '스파게티',
   '432583': '칼국수',
   '432584': '쌀국수',
   '432586': '냉면',
   '432587': '비빔냉면',
   '432590': '쫄면',
   '432593': '수제비',
   '486443': '콩국수',
   '486445': '월남쌈',
   '486846': '즉석파스타',
   '486848': '페투치네',
   '486849': '링귀니',
   '486850': '펜네',
   '486851': '파르펠리',
   '486852': '리가토니',
   '486475': '물냉면',
   '486845': '당면',
   '432821': '면'}},
 {3: {'432790': '떡',
   '432791': '가래떡',
   '432793': '찹쌀떡',
   '432794': '두텁떡',
   '432795': '콩떡',
   '432796': '감자떡',
   '432797': '송편',
   '432798': '백설기',
   '432799': '경단',
   '432800': '오메기떡',
   '432801': '떡케익',
   '225499': '떡볶이떡',
   '225500': '떡국떡',
   '446062': '떡케이크'}},
 {4: {'194642': '콩',
   '194646': '강낭콩',
   '194647': '완두콩',
   '19

In [11]:
category_to_top = {}
for c_items in category_items:
    for k, v in c_items.items():
        for v_k in v.keys():
            category_to_top[v_k] = k

len(category_to_top)

714

In [12]:
df = pd.read_csv('data_coupang.csv')
df

,category_id,category_name,product_id,title,price,main_thumbnail,detail_images,product_url,crawling_dt
0,195010,국산생수,7869554357,"동원샘물 무라벨 생수, 500ml, 20개",4150,https://thumbnail.coupangcdn.com/thumbnails/re...,http://image1.coupangcdn.com/image/vendor_inve...,https://www.coupang.com/vp/products/7869554357...,2026-01-06 17:28
1,497920,탄산수,7419943131,"탐사 아쿠아 스파클링 플레인 무라벨, 6개, 500ml",3290,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/7419943131...,2026-01-06 17:28
2,195080,두유,7852023536,"덴마크 그린덴마크 귀리, 950ml, 1개",2780,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/7852023536...,2026-01-06 17:28
3,195083,검은콩두유,8316219203,"완전두유 검은콩 고칼슘, 190ml, 20개",7770,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/8316219203...,2026-01-06 17:28
4,195084,아몬드두유,7510980660,"그린덴마크 아몬드, 950ml, 2개",5560,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/7510980660...,2026-01-06 17:28
...,...,...,...,...,...,...,...,...,...
718,493037,슈가프리 소스,4562192545,"브래그 로우 언필터드 사과 식초, 473ml, 1개",2380,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/4562192545...,2026-01-07 13:00
719,493038,베이커리 잼,28839478,"마라나타 노 슈가 노 솔트 애디드 피넛 버터 크리미, 454g, 1개",4900,https://thumbnail.coupangcdn.com/thumbnails/re...,http://image1.coupangcdn.com/image/vendor_inve...,https://www.coupang.com/vp/products/28839478?i...,2026-01-07 13:00
720,493087,곤약쌀,7419631986,"냄새제로 자연정 찰곤약쌀, 200g, 1개",1450,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/7419631986...,2026-01-07 13:01
721,493088,곤약젤리,7810003331,"더존건강 더조은 한끼곤약젤리 복숭아 10개입, 1.5kg, 1개",5900,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/7810003331...,2026-01-07 13:01


In [13]:
df['category_id'] = df['category_id'].astype(str).map(category_to_top)
df.dropna(subset=['category_id'], inplace=True)
df['category_id'] = df['category_id'].astype(int)
df

,category_id,category_name,product_id,title,price,main_thumbnail,detail_images,product_url,crawling_dt
0,20,국산생수,7869554357,"동원샘물 무라벨 생수, 500ml, 20개",4150,https://thumbnail.coupangcdn.com/thumbnails/re...,http://image1.coupangcdn.com/image/vendor_inve...,https://www.coupang.com/vp/products/7869554357...,2026-01-06 17:28
1,20,탄산수,7419943131,"탐사 아쿠아 스파클링 플레인 무라벨, 6개, 500ml",3290,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/7419943131...,2026-01-06 17:28
2,23,두유,7852023536,"덴마크 그린덴마크 귀리, 950ml, 1개",2780,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/7852023536...,2026-01-06 17:28
3,23,검은콩두유,8316219203,"완전두유 검은콩 고칼슘, 190ml, 20개",7770,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/8316219203...,2026-01-06 17:28
4,23,아몬드두유,7510980660,"그린덴마크 아몬드, 950ml, 2개",5560,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/7510980660...,2026-01-06 17:28
...,...,...,...,...,...,...,...,...,...
718,29,슈가프리 소스,4562192545,"브래그 로우 언필터드 사과 식초, 473ml, 1개",2380,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/4562192545...,2026-01-07 13:00
719,41,베이커리 잼,28839478,"마라나타 노 슈가 노 솔트 애디드 피넛 버터 크리미, 454g, 1개",4900,https://thumbnail.coupangcdn.com/thumbnails/re...,http://image1.coupangcdn.com/image/vendor_inve...,https://www.coupang.com/vp/products/28839478?i...,2026-01-07 13:00
720,1,곤약쌀,7419631986,"냄새제로 자연정 찰곤약쌀, 200g, 1개",1450,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/7419631986...,2026-01-07 13:01
721,42,곤약젤리,7810003331,"더존건강 더조은 한끼곤약젤리 복숭아 10개입, 1.5kg, 1개",5900,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/7810003331...,2026-01-07 13:01


In [14]:
df = df[['category_id', 'category_name', 'title', 'price', 'main_thumbnail', 'detail_images']]
df

,category_id,category_name,title,price,main_thumbnail,detail_images
0,20,국산생수,"동원샘물 무라벨 생수, 500ml, 20개",4150,https://thumbnail.coupangcdn.com/thumbnails/re...,http://image1.coupangcdn.com/image/vendor_inve...
1,20,탄산수,"탐사 아쿠아 스파클링 플레인 무라벨, 6개, 500ml",3290,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...
2,23,두유,"덴마크 그린덴마크 귀리, 950ml, 1개",2780,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...
3,23,검은콩두유,"완전두유 검은콩 고칼슘, 190ml, 20개",7770,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...
4,23,아몬드두유,"그린덴마크 아몬드, 950ml, 2개",5560,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...
...,...,...,...,...,...,...
718,29,슈가프리 소스,"브래그 로우 언필터드 사과 식초, 473ml, 1개",2380,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...
719,41,베이커리 잼,"마라나타 노 슈가 노 솔트 애디드 피넛 버터 크리미, 454g, 1개",4900,https://thumbnail.coupangcdn.com/thumbnails/re...,http://image1.coupangcdn.com/image/vendor_inve...
720,1,곤약쌀,"냄새제로 자연정 찰곤약쌀, 200g, 1개",1450,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...
721,42,곤약젤리,"더존건강 더조은 한끼곤약젤리 복숭아 10개입, 1.5kg, 1개",5900,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...


In [15]:
df = df.rename(columns={'category_name': 'name'})
df

,category_id,name,title,price,main_thumbnail,detail_images
0,20,국산생수,"동원샘물 무라벨 생수, 500ml, 20개",4150,https://thumbnail.coupangcdn.com/thumbnails/re...,http://image1.coupangcdn.com/image/vendor_inve...
1,20,탄산수,"탐사 아쿠아 스파클링 플레인 무라벨, 6개, 500ml",3290,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...
2,23,두유,"덴마크 그린덴마크 귀리, 950ml, 1개",2780,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...
3,23,검은콩두유,"완전두유 검은콩 고칼슘, 190ml, 20개",7770,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...
4,23,아몬드두유,"그린덴마크 아몬드, 950ml, 2개",5560,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...
...,...,...,...,...,...,...
718,29,슈가프리 소스,"브래그 로우 언필터드 사과 식초, 473ml, 1개",2380,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...
719,41,베이커리 잼,"마라나타 노 슈가 노 솔트 애디드 피넛 버터 크리미, 454g, 1개",4900,https://thumbnail.coupangcdn.com/thumbnails/re...,http://image1.coupangcdn.com/image/vendor_inve...
720,1,곤약쌀,"냄새제로 자연정 찰곤약쌀, 200g, 1개",1450,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...
721,42,곤약젤리,"더존건강 더조은 한끼곤약젤리 복숭아 10개입, 1.5kg, 1개",5900,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...


In [16]:
try:
    df.to_sql(
        name='product',
        con=db.get_bind(),
        if_exists='append',
        index=False,
        method='multi'
    )
    db.commit()
except IntegrityError as e:
    print(f"DB IntegrityError: {e}")
    db.rollback()
except SQLAlchemyError as e:
    print(f"DB Error: {e}")
    db.rollback()
except Exception as e:
    print(f"Error: {e}")
    db.rollback()
finally:
    db.close()

### recipe

In [ ]:
# recipe

In [17]:
df = pd.read_csv('data_10000recipe.csv')
df

,Recipe_Link,Title,Serving,Preparation_Time,Difficulty,Main_Thumbnail,Summary_In,Ingredient,Condiment,Steps
0,https://www.10000recipe.com/recipe/7052506,레드벨벳 슬기님의 컴백 전 관리비결!,1인분,10분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,요 레시피 레드벨벳 슬기님이 자주 해먹으신다는 레시피인데 짝꿍이 먹어보더니 불고기 ...,"두부-1/2모,팽이버섯-1/2봉지","진간장-1T,알룰로스-1T,다진마늘-1T,물-2T,굴소스-0.5T",1단계||두부 반 모 노릇하게 구워서 접시에 올려주세요||https://recipe...
1,https://www.10000recipe.com/recipe/6966445,파프리카달걀볶음:초간단 저탄고지 다이어트 단백질식사,1인분,10분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,토마토가 없어 토달볶대신 만든 파달볶,"파프리카-1개,달걀-3개,양파-,현미유-조금,소금-조금,후추-조금,파슬리가루-",NaN,1단계||파프리카와 양파는 한입 크기로 썬다.||https://recipe1.ezm...
2,https://www.10000recipe.com/recipe/7046232,<다이어트 건강식>초간단 순두부 계란찜,1인분,10분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,*단백질 폭탄 영양 만점~ 한 끼 식사! *불 필요없이 전자레인지로 초간단 레시피,"달걀-2알,순두부-1팩,참기름-1T,참치액-1T,파다진 파-1T",NaN,1단계||순두부 포장 부분 물로 깨끗이 씻어 주세요.||https://recipe1...
3,https://www.10000recipe.com/recipe/6990828,"5분 컷 세상 간단한 달걀요리, 우유달걀카레",1인분,10분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,맛있는 카레에 몽글몽글 달걀을 풀어준부드러운 우유달걀카레달걀과 우유를 섞어 부드럽게...,"달걀-2개,우유-2숟가락,마늘-4개,양파-1/4개,버터-1조각,물-1.5종이컵,카레...",NaN,1단계||마늘은 편 썬다.||https://recipe1.ezmember.co.kr...
4,https://www.10000recipe.com/recipe/6868435,[간단 자취요리] 비주얼 끝장판!! 참치 스테이크 만들기,1인분,30분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,안녕하세요 여러분?? 히히 오늘은 참치 스테이크를 만들어 봅시다!!소스는 간단하게 ...,"참치캔기본사이즈-1개,부침가루-,양파-1/2개,당근-",소금-2/3t,1단계||재료를 준비한다.||https://recipe1.ezmember.co.kr...
...,...,...,...,...,...,...,...,...,...,...
1161,https://www.10000recipe.com/recipe/7058965,다이어터 입맛 저격 / 두부 부추 새우전,1인분,20분 이내,초급,https://recipe1.ezmember.co.kr/cache/recipe/20...,"다이어트 중에도 맛있게 먹자! 기름 적게, 밀가루 없이! 촉촉하고 담백한 두부 부추...","두부-1/2모,부추-1줌,새우-10마리,굴소스-1작은술,계란-3개",NaN,1단계||두부를 으깨어 수분을 날리며 볶아주세요||https://recipe1.ez...
1162,https://www.10000recipe.com/recipe/7059144,초간단 살치살 야채구이,1인분,5분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,초간단 레시피로 고급진 한끼로 살치살 야채구이 만들어 드세요,"살치살-1팩,시즈닝-1봉지,고추-2개,양파-1/3개,버섯-조금,올리브오일-조금,쌈장...",NaN,1단계||달궈진 프라이팬에 살치살을 올려 주세요||https://recipe1.ez...
1163,https://www.10000recipe.com/recipe/7055337,고갈비 고등어구이 양념장 레시피,1인분,60분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,장어구이 대신 저렴한 가격에 즐길 수 있는 밥도둑 생선구이 고갈비 레시피 준비해 보...,"고등어 大-1마리,식용유-2스푼,통깨-약간","설탕-2스푼,고춧가루-1스푼,고추장-1스푼,진간장-1스푼,멸치액젓-1스푼,맛술-2스...",1단계||냉동 생선은 물 1L 기준 밀가루 1스푼을 섞어 30분 동안 담가 놓으세요...
1164,https://www.10000recipe.com/recipe/7055490,한번 만들어 놓으면 밥도둑 ‘소보로 갈비찜’ 레시피 알려드릴게요.,1인분,30분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,"어제 저녁 메뉴로 남은 음식 활용법입니다. 밥에 올려 비벼 먹으면 맛있고, 어린이들...","소고기 다짐육-400g,무-200g,당근-1/2개,표고버섯-2개,양파-1/2개,배음...",NaN,"1단계||무,당근,표고버섯을 준비해주시고, 작은 큐브모양으로 썰어주세요.||http..."


In [18]:
df.drop_duplicates(subset=['Title'], inplace=True, ignore_index=True)
df['Title'].value_counts()

Title
레드벨벳 슬기님의 컴백 전 관리비결!                               1
팽이 김말이                                             1
아이들 아침밥 시금치돼지고기 덮밥 만들기                             1
쫄깃한 식감과 부들부들 맛있는 "동파육"레시피 알려드려요!                   1
돼지 등뼈 비비드키친 바베큐 양념 에어프라이어 구이                       1
                                                  ..
☆ 우삼겹 부추볶음 ☆                                       1
콩불 콩나물불고기 만드는법                                     1
밥반찬으로 좋은 닭가슴살 데리야끼볶음                               1
치즈돼지고기롤...돼지불고기로 품격있게....*                         1
닭가슴살 파프리카 양파 볶음 만드는법 /닭가슴살요리/ 냉동닭가슴살 돈까스소스 야채볶음    1
Name: count, Length: 1165, dtype: int64

In [19]:
df

,Recipe_Link,Title,Serving,Preparation_Time,Difficulty,Main_Thumbnail,Summary_In,Ingredient,Condiment,Steps
0,https://www.10000recipe.com/recipe/7052506,레드벨벳 슬기님의 컴백 전 관리비결!,1인분,10분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,요 레시피 레드벨벳 슬기님이 자주 해먹으신다는 레시피인데 짝꿍이 먹어보더니 불고기 ...,"두부-1/2모,팽이버섯-1/2봉지","진간장-1T,알룰로스-1T,다진마늘-1T,물-2T,굴소스-0.5T",1단계||두부 반 모 노릇하게 구워서 접시에 올려주세요||https://recipe...
1,https://www.10000recipe.com/recipe/6966445,파프리카달걀볶음:초간단 저탄고지 다이어트 단백질식사,1인분,10분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,토마토가 없어 토달볶대신 만든 파달볶,"파프리카-1개,달걀-3개,양파-,현미유-조금,소금-조금,후추-조금,파슬리가루-",NaN,1단계||파프리카와 양파는 한입 크기로 썬다.||https://recipe1.ezm...
2,https://www.10000recipe.com/recipe/7046232,<다이어트 건강식>초간단 순두부 계란찜,1인분,10분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,*단백질 폭탄 영양 만점~ 한 끼 식사! *불 필요없이 전자레인지로 초간단 레시피,"달걀-2알,순두부-1팩,참기름-1T,참치액-1T,파다진 파-1T",NaN,1단계||순두부 포장 부분 물로 깨끗이 씻어 주세요.||https://recipe1...
3,https://www.10000recipe.com/recipe/6990828,"5분 컷 세상 간단한 달걀요리, 우유달걀카레",1인분,10분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,맛있는 카레에 몽글몽글 달걀을 풀어준부드러운 우유달걀카레달걀과 우유를 섞어 부드럽게...,"달걀-2개,우유-2숟가락,마늘-4개,양파-1/4개,버터-1조각,물-1.5종이컵,카레...",NaN,1단계||마늘은 편 썬다.||https://recipe1.ezmember.co.kr...
4,https://www.10000recipe.com/recipe/6868435,[간단 자취요리] 비주얼 끝장판!! 참치 스테이크 만들기,1인분,30분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,안녕하세요 여러분?? 히히 오늘은 참치 스테이크를 만들어 봅시다!!소스는 간단하게 ...,"참치캔기본사이즈-1개,부침가루-,양파-1/2개,당근-",소금-2/3t,1단계||재료를 준비한다.||https://recipe1.ezmember.co.kr...
...,...,...,...,...,...,...,...,...,...,...
1160,https://www.10000recipe.com/recipe/7058965,다이어터 입맛 저격 / 두부 부추 새우전,1인분,20분 이내,초급,https://recipe1.ezmember.co.kr/cache/recipe/20...,"다이어트 중에도 맛있게 먹자! 기름 적게, 밀가루 없이! 촉촉하고 담백한 두부 부추...","두부-1/2모,부추-1줌,새우-10마리,굴소스-1작은술,계란-3개",NaN,1단계||두부를 으깨어 수분을 날리며 볶아주세요||https://recipe1.ez...
1161,https://www.10000recipe.com/recipe/7059144,초간단 살치살 야채구이,1인분,5분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,초간단 레시피로 고급진 한끼로 살치살 야채구이 만들어 드세요,"살치살-1팩,시즈닝-1봉지,고추-2개,양파-1/3개,버섯-조금,올리브오일-조금,쌈장...",NaN,1단계||달궈진 프라이팬에 살치살을 올려 주세요||https://recipe1.ez...
1162,https://www.10000recipe.com/recipe/7055337,고갈비 고등어구이 양념장 레시피,1인분,60분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,장어구이 대신 저렴한 가격에 즐길 수 있는 밥도둑 생선구이 고갈비 레시피 준비해 보...,"고등어 大-1마리,식용유-2스푼,통깨-약간","설탕-2스푼,고춧가루-1스푼,고추장-1스푼,진간장-1스푼,멸치액젓-1스푼,맛술-2스...",1단계||냉동 생선은 물 1L 기준 밀가루 1스푼을 섞어 30분 동안 담가 놓으세요...
1163,https://www.10000recipe.com/recipe/7055490,한번 만들어 놓으면 밥도둑 ‘소보로 갈비찜’ 레시피 알려드릴게요.,1인분,30분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,"어제 저녁 메뉴로 남은 음식 활용법입니다. 밥에 올려 비벼 먹으면 맛있고, 어린이들...","소고기 다짐육-400g,무-200g,당근-1/2개,표고버섯-2개,양파-1/2개,배음...",NaN,"1단계||무,당근,표고버섯을 준비해주시고, 작은 큐브모양으로 썰어주세요.||http..."


In [20]:
df['Ingredient'] = df['Ingredient'].str.replace(r'-[^,]*', '', regex=True)
df['Ingredient']

0                                                 두부,팽이버섯
1                              파프리카,달걀,양파,현미유,소금,후추,파슬리가루
2                                    달걀,순두부,참기름,참치액,파다진 파
3                                달걀,우유,마늘,양파,버터,물,카레가루,소금
4                                     참치캔기본사이즈,부침가루,양파,당근
                              ...                        
1160                                      두부,부추,새우,굴소스,계란
1161                     살치살,시즈닝,고추,양파,버섯,올리브오일,쌈장와사비보리쌈장
1162                                         고등어 大,식용유,통깨
1163    소고기 다짐육,무,당근,표고버섯,양파,배음료,간장,자일로스 설탕,맛술,후추소고기 볶...
1164    닭가슴살,양파,빨간 파프리카,노란파프리카,스테이크소스돈까스소스 대체가능,올리브오일,...
Name: Ingredient, Length: 1165, dtype: object

In [35]:
df_recipe = df[['Title', 'Ingredient', 'Preparation_Time', 'Main_Thumbnail']]
df_recipe = df_recipe.rename(columns={'Title': 'name', 'Ingredient': 'ingredient', 'Preparation_Time': 'time', 'Main_Thumbnail': 'thumbnail'})
df_recipe

,name,ingredient,time,thumbnail
0,레드벨벳 슬기님의 컴백 전 관리비결!,"두부,팽이버섯",10분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...
1,파프리카달걀볶음:초간단 저탄고지 다이어트 단백질식사,"파프리카,달걀,양파,현미유,소금,후추,파슬리가루",10분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...
2,<다이어트 건강식>초간단 순두부 계란찜,"달걀,순두부,참기름,참치액,파다진 파",10분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...
3,"5분 컷 세상 간단한 달걀요리, 우유달걀카레","달걀,우유,마늘,양파,버터,물,카레가루,소금",10분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...
4,[간단 자취요리] 비주얼 끝장판!! 참치 스테이크 만들기,"참치캔기본사이즈,부침가루,양파,당근",30분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...
...,...,...,...,...
1160,다이어터 입맛 저격 / 두부 부추 새우전,"두부,부추,새우,굴소스,계란",20분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...
1161,초간단 살치살 야채구이,"살치살,시즈닝,고추,양파,버섯,올리브오일,쌈장와사비보리쌈장",5분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...
1162,고갈비 고등어구이 양념장 레시피,"고등어 大,식용유,통깨",60분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...
1163,한번 만들어 놓으면 밥도둑 ‘소보로 갈비찜’ 레시피 알려드릴게요.,"소고기 다짐육,무,당근,표고버섯,양파,배음료,간장,자일로스 설탕,맛술,후추소고기 볶...",30분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...


In [23]:
try:
    df_recipe.to_sql(
        name='recipe',
        con=db.get_bind(),
        if_exists='append',
        index=False,
        method='multi'
    )
    db.commit()
except IntegrityError as e:
    print(f"DB IntegrityError: {e}")
    db.rollback()
except SQLAlchemyError as e:
    print(f"DB Error: {e}")
    db.rollback()
except Exception as e:
    print(f"Error: {e}")
    db.rollback()
finally:
    db.close()

In [ ]:
# recipe_product

In [52]:
df_recipe['recipe_id'] = range(1, len(df_recipe) + 1)
df_recipe['recipe_id']

0          1
1          2
2          3
3          4
4          5
        ... 
1160    1161
1161    1162
1162    1163
1163    1164
1164    1165
Name: recipe_id, Length: 1165, dtype: int64

In [53]:
df_recipe['ingredient'] = df_recipe['ingredient'].str.split(',')
df_recipe

,name,ingredient,time,thumbnail,recipe_id
0,레드벨벳 슬기님의 컴백 전 관리비결!,NaN,10분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...,1
1,파프리카달걀볶음:초간단 저탄고지 다이어트 단백질식사,NaN,10분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...,2
2,<다이어트 건강식>초간단 순두부 계란찜,NaN,10분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...,3
3,"5분 컷 세상 간단한 달걀요리, 우유달걀카레",NaN,10분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...,4
4,[간단 자취요리] 비주얼 끝장판!! 참치 스테이크 만들기,NaN,30분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...,5
...,...,...,...,...,...
1160,다이어터 입맛 저격 / 두부 부추 새우전,NaN,20분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...,1161
1161,초간단 살치살 야채구이,NaN,5분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...,1162
1162,고갈비 고등어구이 양념장 레시피,NaN,60분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...,1163
1163,한번 만들어 놓으면 밥도둑 ‘소보로 갈비찜’ 레시피 알려드릴게요.,NaN,30분 이내,https://recipe1.ezmember.co.kr/cache/recipe/20...,1164


In [54]:
df_recipe_product = df_recipe.explode('ingredient', ignore_index=True)
df_recipe_product = df_recipe_product[['recipe_id', 'ingredient']]
df_recipe_product

,recipe_id,ingredient
0,1,NaN
1,2,NaN
2,3,NaN
3,4,NaN
4,5,NaN
...,...,...
1160,1161,NaN
1161,1162,NaN
1162,1163,NaN
1163,1164,NaN


In [39]:
try:
    df_recipe_product.to_sql(
        name='recipe_product',
        con=db.get_bind(),
        if_exists='append',
        index=False,
        method='multi'
    )
    db.commit()
except IntegrityError as e:
    print(f"DB IntegrityError: {e}")
    db.rollback()
except SQLAlchemyError as e:
    print(f"DB Error: {e}")
    db.rollback()
except Exception as e:
    print(f"Error: {e}")
    db.rollback()
finally:
    db.close()

In [ ]:
# recipe_step

In [60]:
df_recipe_step = df_recipe[['recipe_id']].copy()
df_recipe_step['raw'] = df['Steps'].str.split('\n')
df_recipe_step

,recipe_id,raw
0,1,[1단계||두부 반 모 노릇하게 구워서 접시에 올려주세요||https://recip...
1,2,[1단계||파프리카와 양파는 한입 크기로 썬다.||https://recipe1.ez...
2,3,[1단계||순두부 포장 부분 물로 깨끗이 씻어 주세요.||https://recipe...
3,4,[1단계||마늘은 편 썬다.||https://recipe1.ezmember.co.k...
4,5,[1단계||재료를 준비한다.||https://recipe1.ezmember.co.k...
...,...,...
1160,1161,[1단계||두부를 으깨어 수분을 날리며 볶아주세요||https://recipe1.e...
1161,1162,[1단계||달궈진 프라이팬에 살치살을 올려 주세요||https://recipe1.e...
1162,1163,[1단계||냉동 생선은 물 1L 기준 밀가루 1스푼을 섞어 30분 동안 담가 놓으세...
1163,1164,"[1단계||무,당근,표고버섯을 준비해주시고, 작은 큐브모양으로 썰어주세요.||htt..."


In [62]:
df_recipe_step = df_recipe_step.explode('raw', ignore_index=True)
df_recipe_step

,recipe_id,raw
0,1,1단계||두부 반 모 노릇하게 구워서 접시에 올려주세요||https://recipe...
1,1,2단계||팽이버섯 반 봉지 구워주다가||https://recipe1.ezmember...
2,1,"3단계||진간장1T, 알룰로스1T, 굴소스0.5T, 다진마늘1T, 물2T 섞어서 소..."
3,1,4단계||두부 위에 올려주면 끝!||https://recipe1.ezmember.c...
4,2,1단계||파프리카와 양파는 한입 크기로 썬다.||https://recipe1.ezm...
...,...,...
9004,1165,"4단계||팬에 올리브오일 2-3T, 닭 가슴살 넣고 구워줍니다.||https://r..."
9005,1165,"5단계||닭이 거의 다 구워졌을 때 양파, 파프리카, 후추 적당량, 스테이크 소스 ..."
9006,1165,6단계||채소가 익을 때까지 구워주세요.||https://recipe1.ezmemb...
9007,1165,"7단계||완성된 요리를 접시로 옮긴 후, 파슬리 가루 뿌려 마무리해 줍니다.||ht..."


In [ ]:
# recipe_id, step_number, description, url

In [68]:
split_raw_df = df_recipe_step['raw'].str.split('||', regex=False, expand=True)
split_raw_df = split_raw_df.rename(columns={0: "step_number", 1: "description", 2: "url"})
split_raw_df

,step_number,description,url
0,1단계,두부 반 모 노릇하게 구워서 접시에 올려주세요,https://recipe1.ezmember.co.kr/cache/recipe/20...
1,2단계,팽이버섯 반 봉지 구워주다가,https://recipe1.ezmember.co.kr/cache/recipe/20...
2,3단계,"진간장1T, 알룰로스1T, 굴소스0.5T, 다진마늘1T, 물2T 섞어서 소스만들어 ...",https://recipe1.ezmember.co.kr/cache/recipe/20...
3,4단계,두부 위에 올려주면 끝!,https://recipe1.ezmember.co.kr/cache/recipe/20...
4,1단계,파프리카와 양파는 한입 크기로 썬다.,https://recipe1.ezmember.co.kr/cache/recipe/20...
...,...,...,...
9004,4단계,"팬에 올리브오일 2-3T, 닭 가슴살 넣고 구워줍니다.",https://recipe1.ezmember.co.kr/cache/recipe/20...
9005,5단계,"닭이 거의 다 구워졌을 때 양파, 파프리카, 후추 적당량, 스테이크 소스 3-4T ...",https://recipe1.ezmember.co.kr/cache/recipe/20...
9006,6단계,채소가 익을 때까지 구워주세요.,https://recipe1.ezmember.co.kr/cache/recipe/20...
9007,7단계,"완성된 요리를 접시로 옮긴 후, 파슬리 가루 뿌려 마무리해 줍니다.",https://recipe1.ezmember.co.kr/cache/recipe/20...


In [69]:
df_recipe_step_final = pd.concat([df_recipe_step['recipe_id'], split_raw_df], axis=1)
df_recipe_step_final

,recipe_id,step_number,description,url
0,1,1단계,두부 반 모 노릇하게 구워서 접시에 올려주세요,https://recipe1.ezmember.co.kr/cache/recipe/20...
1,1,2단계,팽이버섯 반 봉지 구워주다가,https://recipe1.ezmember.co.kr/cache/recipe/20...
2,1,3단계,"진간장1T, 알룰로스1T, 굴소스0.5T, 다진마늘1T, 물2T 섞어서 소스만들어 ...",https://recipe1.ezmember.co.kr/cache/recipe/20...
3,1,4단계,두부 위에 올려주면 끝!,https://recipe1.ezmember.co.kr/cache/recipe/20...
4,2,1단계,파프리카와 양파는 한입 크기로 썬다.,https://recipe1.ezmember.co.kr/cache/recipe/20...
...,...,...,...,...
9004,1165,4단계,"팬에 올리브오일 2-3T, 닭 가슴살 넣고 구워줍니다.",https://recipe1.ezmember.co.kr/cache/recipe/20...
9005,1165,5단계,"닭이 거의 다 구워졌을 때 양파, 파프리카, 후추 적당량, 스테이크 소스 3-4T ...",https://recipe1.ezmember.co.kr/cache/recipe/20...
9006,1165,6단계,채소가 익을 때까지 구워주세요.,https://recipe1.ezmember.co.kr/cache/recipe/20...
9007,1165,7단계,"완성된 요리를 접시로 옮긴 후, 파슬리 가루 뿌려 마무리해 줍니다.",https://recipe1.ezmember.co.kr/cache/recipe/20...


In [73]:
df_recipe_step_final['step_number'] = df_recipe_step_final['step_number'].str.replace('단계', '')
df_recipe_step_final

,recipe_id,step_number,description,url
0,1,1,두부 반 모 노릇하게 구워서 접시에 올려주세요,https://recipe1.ezmember.co.kr/cache/recipe/20...
1,1,2,팽이버섯 반 봉지 구워주다가,https://recipe1.ezmember.co.kr/cache/recipe/20...
2,1,3,"진간장1T, 알룰로스1T, 굴소스0.5T, 다진마늘1T, 물2T 섞어서 소스만들어 ...",https://recipe1.ezmember.co.kr/cache/recipe/20...
3,1,4,두부 위에 올려주면 끝!,https://recipe1.ezmember.co.kr/cache/recipe/20...
4,2,1,파프리카와 양파는 한입 크기로 썬다.,https://recipe1.ezmember.co.kr/cache/recipe/20...
...,...,...,...,...
9004,1165,4,"팬에 올리브오일 2-3T, 닭 가슴살 넣고 구워줍니다.",https://recipe1.ezmember.co.kr/cache/recipe/20...
9005,1165,5,"닭이 거의 다 구워졌을 때 양파, 파프리카, 후추 적당량, 스테이크 소스 3-4T ...",https://recipe1.ezmember.co.kr/cache/recipe/20...
9006,1165,6,채소가 익을 때까지 구워주세요.,https://recipe1.ezmember.co.kr/cache/recipe/20...
9007,1165,7,"완성된 요리를 접시로 옮긴 후, 파슬리 가루 뿌려 마무리해 줍니다.",https://recipe1.ezmember.co.kr/cache/recipe/20...


In [77]:
df_recipe_step_final['step_number'][df_recipe_step_final['step_number'].isna()]

8867    NaN
Name: step_number, dtype: object

In [83]:
df_recipe_step_final.dropna(inplace=True)
df_recipe_step_final['step_number'] = df_recipe_step_final['step_number'].astype(int)
df_recipe_step_final

,recipe_id,step_number,description,url
0,1,1,두부 반 모 노릇하게 구워서 접시에 올려주세요,https://recipe1.ezmember.co.kr/cache/recipe/20...
1,1,2,팽이버섯 반 봉지 구워주다가,https://recipe1.ezmember.co.kr/cache/recipe/20...
2,1,3,"진간장1T, 알룰로스1T, 굴소스0.5T, 다진마늘1T, 물2T 섞어서 소스만들어 ...",https://recipe1.ezmember.co.kr/cache/recipe/20...
3,1,4,두부 위에 올려주면 끝!,https://recipe1.ezmember.co.kr/cache/recipe/20...
4,2,1,파프리카와 양파는 한입 크기로 썬다.,https://recipe1.ezmember.co.kr/cache/recipe/20...
...,...,...,...,...
9004,1165,4,"팬에 올리브오일 2-3T, 닭 가슴살 넣고 구워줍니다.",https://recipe1.ezmember.co.kr/cache/recipe/20...
9005,1165,5,"닭이 거의 다 구워졌을 때 양파, 파프리카, 후추 적당량, 스테이크 소스 3-4T ...",https://recipe1.ezmember.co.kr/cache/recipe/20...
9006,1165,6,채소가 익을 때까지 구워주세요.,https://recipe1.ezmember.co.kr/cache/recipe/20...
9007,1165,7,"완성된 요리를 접시로 옮긴 후, 파슬리 가루 뿌려 마무리해 줍니다.",https://recipe1.ezmember.co.kr/cache/recipe/20...


In [84]:
try:
    df_recipe_step_final.to_sql(
        name='recipe_step',
        con=db.get_bind(),
        if_exists='append',
        index=False,
        method='multi'
    )
    db.commit()
except IntegrityError as e:
    print(f"DB IntegrityError: {e}")
    db.rollback()
except SQLAlchemyError as e:
    print(f"DB Error: {e}")
    db.rollback()
except Exception as e:
    print(f"Error: {e}")
    db.rollback()
finally:
    db.close()